<a href="https://colab.research.google.com/github/HAV-79/Weather-Forecast/blob/main/Classifica%C3%A7%C3%A3o_do_tempo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Funcionando 100% no VSCode

In [ ]:
import os
import tensorflow as tf
import tensorflow.keras as keras
from keras import Sequential
from tensorflow.keras.preprocessing.image import img_to_array, img_to_array, load_img
from tensorflow.python.client import device_lib
from tensorflow.keras.utils import model_to_dot
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model


from IPython.display import SVG
from PIL import Image
import seaborn as sns

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from os import mkdir

import scipy

tf.test.gpu_device_name()
device_lib.list_local_devices()

if tf.config.list_physical_devices('GPU'):
   print('GPU disponível')

else:
   print('GPU não disponível')

# tf.test.gpu_device_name()
# device_lib.list_local_devices()

# from google.colab import drive
# drive.mount('/content/drive')

path = 'I:/Meu Drive/dataset/'

lista = os.listdir(path)

print(lista)

#neste ponto vamos contar todos os arquivos e mostrar a estrutura de subdiretórios
totalFiles = 0
totalDir = 0
PATH = path

for base, dirs, files in os.walk(PATH):
    print('Searching in : ',base)
    for directories in dirs:
        totalDir += 1
    for Files in files:
        totalFiles += 1
print('Total number of files',totalFiles)
print('Total Number of directories',totalDir)
print('Total:',(totalDir + totalFiles))

#contamos as amostras de imagens para cada categoria.

targets = ['nublado','chuva','sol']

def count_targets(path_dataset, targets = targets):
  """
  Função para contar amostras para cada categoria
  entrada:
    caminho para conjunto de dados - (exemplo, validação, treinamento ou teste)
    alvos - alvos do conjunto de dados para cada categoria
  saída:
    alvo de contagem - matriz numpy com arquivos de contagem para cada categoria

  """
  counts_target = []
  for folder in targets:
    path, dirs, files = next(os.walk(path_dataset+folder))
    file_count = len(files)
    counts_target.append([folder,file_count])

  return np.array(counts_target, dtype = object)

train_samples = count_targets(path_dataset = 'I:/Meu Drive/dataset/training/')
validation_samples = count_targets(path_dataset = 'I:/Meu Drive/dataset/validation/')
test_samples = count_targets(path_dataset = 'I:/Meu Drive/dataset/test/')


#vamos plotar a distribuição de amostras para treinamento
fig = plt.figure(figsize = (10,5))
ax = sns.barplot(x = train_samples[:,0], y = train_samples[:,1])
ax.set_title("Training Samples")


#vamos plotar a distribuição de amostras para validação
fig = plt.figure(figsize = (10,5))
ax = sns.barplot(x = train_samples[:,0], y = validation_samples[:,1])
ax.set_title("Validation Samples")


#vamos plotar a distribuição de amostras para teste
fig = plt.figure(figsize = (10,5))
ax = sns.barplot(x = test_samples[:,0], y = test_samples[:,1])
ax.set_title("Test Samples")

#Amostras de imagens

#nublado
img = Image.open("I:/Meu Drive/dataset/training/nublado/nublado_6.jpg")
print(img.size)
print(plt.imshow(img))

#chuva
img = Image.open("I:/Meu Drive/dataset/training/chuva/chuva_1.jpg")
print(img.size)
print(plt.imshow(img))

#sol
img = Image.open("I:/Meu Drive/dataset/training/sol/sol_1.jpg")
print(img.size)
print(plt.imshow(img))


#03 - Processamento de imagem e aumento de dados

# mkdir("I:/Meu Drive/preview")
# from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# datagen = ImageDataGenerator(
#         rotation_range=30,
#         width_shift_range=0.2,
#         height_shift_range=0.2,
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True,
#         fill_mode='nearest')

# img = load_img('I:/Meu Drive/dataset/training/chuva/chuva_10.jpg')
# x = img_to_array(img)
# x = x.reshape((1,) + x.shape)

# # o comando .flow() abaixo gera lotes de imagens transformadas aleatoriamente
# # e salva os resultados no diretório `preview/`
# i = 0
# for batch in datagen.flow(x, batch_size=1,
#                           save_to_dir='preview', save_prefix='rain', save_format='jpg'):
#     i += 1
#     if i > 20:
#         break


BATCH_SIZE = 32
IMAGE_SIZE = 150

# esta é a configuração de aumento que usaremos apenas para treinamento
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# esta é a configuração de aumento que usaremos para validação:
# apenas para redimensionar
validation_datagen = ImageDataGenerator(rescale=1./255)

# esta é a configuração de aumento que usaremos para testar:
# apenas para redimensionar
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'I:/Meu Drive/dataset/training',
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical')

# este é um gerador semelhante, para dados de validação
validation_generator = validation_datagen.flow_from_directory(
        'I:/Meu Drive/dataset/validation/',
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical')

# este é um gerador semelhante, para dados de teste
test_generator = test_datagen.flow_from_directory(
        'I:/Meu Drive/dataset/test',
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical')


#04 - Implementação do modelo

keras.backend.clear_session()

input_layer = keras.layers.Input(shape=(IMAGE_SIZE,IMAGE_SIZE,3))

#CNN - Convolutional Layers (Camadas Convolucionais)
conv_layer = keras.layers.Conv2D(32, (3,3), activation="relu", padding="same")(input_layer)
conv_layer = keras.layers.Conv2D(32, (3,3), activation="relu", padding="same")(conv_layer)
pooling_layer = keras.layers.MaxPool2D(pool_size=(2,2))(conv_layer)

conv_layer = keras.layers.Conv2D(64, (3,3), activation="relu", padding="same")(pooling_layer)
conv_layer = keras.layers.Conv2D(64, (3,3), activation="relu", padding="same")(conv_layer)
pooling_layer = keras.layers.MaxPool2D(pool_size=(2,2))(conv_layer)

#FC - Fully connected (Totalmente conectado)
flatten = keras.layers.Flatten()(pooling_layer)
dense = keras.layers.Dense(200, activation="relu")(flatten)
dropout = keras.layers.Dropout(0.5)(dense)
dense = keras.layers.Dense(100, activation="relu")(dropout)
dropout = keras.layers.Dropout(0.2)(dense)

classifier = keras.layers.Dense(3, activation="softmax")(dropout)

model = keras.Model(inputs=input_layer, outputs=classifier)
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss="categorical_crossentropy",metrics=['accuracy'])
model.summary()


# if COLAB:
#     display(SVG(model_to_dot(model, show_shapes=True,dpi=70).create(prog='dot', format='svg')))
# else:
#    display(SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg')))


# pip install visualkeras (Instalar no VScode do PC)
import visualkeras

visualkeras.layered_view(model).show() # exibir usando o visualizador do sistema
visualkeras.layered_view(model, legend=True,scale_xy=2)

# 05 - Modelo de Treinamento
history = model.fit(train_generator,validation_data=validation_generator, epochs=1300)

# Salvando o modelo treinado
model.save('I:/Meu Drive/model_cnn_save/hav_cnn_epochs_1300.h5')

# 06 - Plotagem de Perda e Precisão

# import matplotlib.pyplot as plt
def plot_curves(history):
  plt.figure()
  plt.xlabel('Epochs')
  plt.ylabel('Error')
  plt.plot(history['loss'])
  plt.plot(history['val_loss'])
  plt.legend(['Training', 'Validation'])

  plt.figure()
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.plot(history['accuracy'])
  plt.plot(history['val_accuracy'])
  plt.legend(['Training', 'Validation'], loc='lower right');


# Desempenho do modelo de plotagem

plot_curves(history.history)
plt.show()


# 07 - Pontuações preliminares (treinamento e validação)

scores_tr = model.evaluate(train_generator)
print('Train loss    :', scores_tr[0])
print('Train accuracy:', scores_tr[1])

scores_val = model.evaluate(validation_generator)
print('Val loss    :', scores_val[0])
print('Val accuracy:', scores_val[1])

scores_tr = model.evaluate(test_generator)
print('Test loss    :', scores_tr[0])
print('Test accuracy:', scores_tr[1])


# 09 - Notas Finais (Teste)

scores_tr = model.evaluate(test_generator)
print('Test loss    :', scores_tr[0])
print('Test accuracy:', scores_tr[1])


# 10 - Previsões

#imprimir índices das classes
indices = train_generator.class_indices
print(indices)


#  Previsões usando o dataset


#previsão para amostras de chuva
path = 'I:/Meu Drive/dataset/test/chuva/chuva_346.jpg'
img=load_img(path, target_size=(IMAGE_SIZE, IMAGE_SIZE))
x=img_to_array(img)
x=np.expand_dims(x, axis=0)
images = np.vstack([x])
classes = model.predict(images)
plt.imshow(img)

max_value = max(classes[0])
max_index = list(classes[0]).index(max_value)

np.around(classes[0], decimals = 2)
print('Real: Chuva', ',Predicted:',list(indices.keys())[max_index])


#previsão para amostras de nublado
path = 'I:/Meu Drive/dataset/test/nublado/nublado_234.jpg'
img=load_img(path, target_size=(IMAGE_SIZE, IMAGE_SIZE))
x=img_to_array(img)
x=np.expand_dims(x, axis=0)
images = np.vstack([x])
classes = model.predict(images)
plt.imshow(img)

max_value = max(classes[0])
max_index = list(classes[0]).index(max_value)

np.around(classes[0], decimals = 2)
print('Real: Nublado', ',Predicted:',list(indices.keys())[max_index])


#previsão para amostras do nascer do sol
path = 'I:/Meu Drive/dataset/test/sol/sol_122.jpg'
img=load_img(path, target_size=(IMAGE_SIZE, IMAGE_SIZE))
x=img_to_array(img)
x=np.expand_dims(x, axis=0)
images = np.vstack([x])
classes = model.predict(images)
plt.imshow(img)

max_value = max(classes[0])
max_index = list(classes[0]).index(max_value)

np.around(classes[0], decimals = 2)
print('Real: Sol', ',Predicted:',list(indices.keys())[max_index])